In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from fedbiomed.common.training_plans import TorchTrainingPlan
from fedbiomed.common.data import DataManager
import pandas as pd
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

# Dataset personalizado que carga CSV desde la ruta local del nodo
class BreastCancerDataset(Dataset):
    def __init__(self, csv_path):
        df = pd.read_csv(csv_path)
        # Asumiendo que las columnas de características están en df.columns[:-1]
        # y la última columna es la etiqueta
        X = df.iloc[:, :-1].values.astype(np.float32)
        y = df.iloc[:, -1].values.astype(np.int64)

        # Normalizar características
        scaler = StandardScaler()
        X = scaler.fit_transform(X)

        self.X = torch.tensor(X)
        self.y = torch.tensor(y)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

class MyTrainingPlan(TorchTrainingPlan):

    class BreastCancerDataset(torch.utils.data.Dataset):
        def __init__(self, csv_path):
            import pandas as pd
            from sklearn.preprocessing import StandardScaler
            df = pd.read_csv(csv_path)
            X = df.iloc[:, :-1].values.astype(np.float32)
            y = df.iloc[:, -1].values.astype(np.int64)
            scaler = StandardScaler()
            X = scaler.fit_transform(X)
            self.X = torch.tensor(X)
            self.y = torch.tensor(y)

        def __len__(self):
            return len(self.y)

        def __getitem__(self, idx):
            return self.X[idx], self.y[idx]

    def init_model(self, model_args):
        class Net(nn.Module):
            def __init__(self):
                super().__init__()
                self.fc1 = nn.Linear(539, 16)
                self.fc2 = nn.Linear(16, 2)

            def forward(self, x):
                x = F.relu(self.fc1(x))
                x = self.fc2(x)
                return F.log_softmax(x, dim=1)
        return Net()

    def init_optimizer(self, optimizer_args):
        return torch.optim.Adam(self.model().parameters(), lr=optimizer_args['lr'])

    def init_dependencies(self):
        return [
            "import numpy as np",
            'import pandas as pd',
            'from sklearn.preprocessing import StandardScaler',
            'import torch',
            'import torch.nn.functional as F'
        ]

    def training_data(self):
        # Usa la clase anidada
        dataset = self.BreastCancerDataset(self.dataset_path)
        return DataManager(dataset=dataset, shuffle=True)

    def training_step(self, data, target):
        output = self.model().forward(data)
        loss = F.nll_loss(output, target)
        return loss

In [2]:
from fedbiomed.common.metrics import MetricTypes

model_args = {}

training_args = {
    'loader_args': {'batch_size': 32},
    'optimizer_args': {'lr': 1e-3},
    'epochs': 5,
    'dry_run': False,
    'batch_maxnum': 100,
    # otros parámetros opcionales
}

In [3]:
from fedbiomed.researcher.federated_workflows import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

tags = ['breast', 'cancer','mamography']  # los tags con los que etiquetaste tu dataset en los nodos
rounds = 2

exp = Experiment(
    tags=tags,
    model_args=model_args,
    training_plan_class=MyTrainingPlan,
    training_args=training_args,
    round_limit=rounds,
    aggregator=FedAverage(),
    tensorboard=True
)

exp.run()  # corre el entrenamiento federado

2025-06-04 21:22:57,988 fedbiomed INFO - Starting researcher service...

2025-06-04 21:22:58,003 fedbiomed INFO - Waiting 3s for nodes to connect...

2025-06-04 21:22:58,756 fedbiomed DEBUG - Node: NODE_c4d7343b-8cc2-4c1e-9543-90dae1f49cff polling for the tasks

2025-06-04 21:22:59,901 fedbiomed INFO - Updating training data. This action will update FederatedDataset, and the nodes that will participate to the experiment.

2025-06-04 21:22:59,950 fedbiomed DEBUG - Node: NODE_c4d7343b-8cc2-4c1e-9543-90dae1f49cff polling for the tasks

...

2025-06-04 21:23:01,638 fedbiomed INFO - TRAINING 
					 NODE_ID: NODE_c4d7343b-8cc2-4c1e-9543-90dae1f49cff 
					 Round 2 Epoch: 5 | Iteration: 1/7 (14%) | Samples: 32/224
 					 Loss: 0.073643 
					 ---------

2025-06-04 21:23:01,646 fedbiomed INFO - TRAINING 
					 NODE_ID: NODE_c4d7343b-8cc2-4c1e-9543-90dae1f49cff 
					 Round 2 Epoch: 5 | Iteration: 2/7 (29%) | Samples: 64/224
 					 Loss: 0.048986 
					 ---------

2025-06-04 21:23:01,663 fedbiomed INFO - TRAINING 
					 NODE_ID: NODE_c4d7343b-8cc2-4c1e-9543-90dae1f49cff 
					 Round 2 Epoch: 5 | Iteration: 7/7 (100%) | Samples: 208/208
 					 Loss: 0.069071 
					 ---------

2025-06-04 21:23:01,680 fedbiomed INFO - Nodes that successfully reply in round 1 ['NODE_c4d7343b-8cc2-4c1e-9543-90dae1f49cff']

2

In [4]:
try:
    exp.training_plan().export_model('./trained_model')
except Exception as e:
    print(e)

print("Training rounds keys:", exp.training_replies().keys())
print("Aggregated params keys:", exp.aggregated_params().keys())

Training rounds keys: dict_keys([0, 1])
Aggregated params keys: dict_keys([0, 1])
